## $${Autoencoder}$$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import cv2

In [2]:
# Funciones
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

In [3]:
# Parámetros
x = np.array([[3, 127, 211],
              [5, 200, 255],
              [29, 8, 10]])

x_normalize = x / 255

In [4]:
# Parámetros de la red
bias = 2 * np.random.random((1, 1)) - 1
input_a = x.shape[1]  # 3 neuronas de entrada (por las 3 características del arreglo) devolvera el tamaño de las 3 columnas
hidden = 2  # 2 neuronas en la capa oculta
output = input_a # Debe ser igual al número de neuronas de entrada en un autoencoder
learning_rate = 0.95
alpha = 0.65  # momentum
epochs = 40000
w_old = 0
b_old = 0

In [5]:
# Inicializar los pesos y bias con valores pequeños aleatorios
np.random.seed(42)  # Fijamos la semilla para reproducibilidad

# Inicialización de pesos y bias
weights_e = np.random.randn(input_a, hidden)
bias_hidden_e = np.random.randn(hidden)

weights_h = np.random.randn(hidden, hidden)
bias_hidden_h = np.random.randn(hidden)

weights_d = np.random.randn(hidden, output)
bias_output = np.random.randn(output)

# Inicialización de variables para momentum
v_weights_e = np.zeros_like(weights_e)
v_bias_hidden_e = np.zeros_like(bias_hidden_e)

v_weights_h = np.zeros_like(weights_h)
v_bias_hidden_h = np.zeros_like(bias_hidden_h)

v_weights_d = np.zeros_like(weights_d)
v_bias_output = np.zeros_like(bias_output)

In [6]:
# Listas para guardar errores
errors_Au_momentum = []

# Entrenamiento con Backpropagation y momentum
for epoch in range(epochs):
    # Forward pass - Encoder
    z_e = np.dot(x_normalize, weights_e) + bias_hidden_e
    yo_e = sigmoid(z_e)

    # Forward pass - Hidden
    z_h = np.dot(yo_e, weights_h) + bias_hidden_h
    yo_h = sigmoid(z_h)

    # Forward pass - Decoder
    z_d = np.dot(yo_h, weights_d) + bias_output
    yo_d = sigmoid(z_d)

    # Calculo del error
    error_Au = x_normalize - yo_d
    mse_Au = np.mean(error_Au**2)
    errors_Au_momentum.append(mse_Au)

    # Backpropagation - Decoder
    d_output = error_Au * sigmoid_derivative(yo_d)
    d_weights_d = np.dot(yo_h.T, d_output)
    db_output = np.sum(d_output, axis=0, keepdims=True)

    # Backpropagation - Hidden
    d_hidden = np.dot(d_output, weights_d.T) * sigmoid_derivative(yo_h)
    d_weights_h = np.dot(yo_e.T, d_hidden)
    db_hidden = np.sum(d_hidden, axis=0, keepdims=True)

    # Backpropagation - Encoder
    d_encoder = np.dot(d_hidden, weights_h.T) * sigmoid_derivative(yo_e)
    d_weights_e = np.dot(x_normalize.T, d_encoder)
    db_encoder = np.sum(d_encoder, axis=0, keepdims=True)

    # Actualización de pesos y sesgos con momentum
    v_weights_d = alpha * (v_weights_d - w_old) + learning_rate * d_weights_d
    v_bias_output = alpha * v_bias_output + learning_rate * db_output.ravel()
    v_weights_e = alpha * (v_weights_e - w_old) + learning_rate * d_weights_e
    v_bias_hidden_e = alpha * v_bias_hidden_e + learning_rate * db_encoder.ravel()
    v_weights_h = alpha * (v_weights_h - w_old) + learning_rate * d_weights_h
    v_bias_hidden_h = alpha * v_bias_hidden_h + learning_rate * db_hidden.ravel()

    # Actualización de los pesos y sesgos
    weights_e += v_weights_e
    bias_hidden_e += v_bias_hidden_e

    weights_h += v_weights_h
    bias_hidden_h += v_bias_hidden_h

    weights_d += v_weights_d
    bias_output += v_bias_output

    #print(weights_d, weights_e)

In [7]:
print(yo_h)

[[0.78782699 0.26465464]
 [0.23196136 0.02017366]
 [0.98929001 0.94493578]]


In [8]:
# Reconstrucción después del entrenamiento
z_e2 = np.dot(x_normalize, weights_e) + bias_hidden_e
a_e2 = sigmoid(z_e2)

z_h2 = np.dot(a_e2, weights_h) + bias_hidden_h
a_h2 = sigmoid(z_h2)

z_d2 = np.dot(a_h2, weights_d) + bias_output
a_d2 = sigmoid(z_d2)

reconstructed_backprop = sigmoid(z_d2)
rounded_reconstructed = np.round(reconstructed_backprop * 255)
print(rounded_reconstructed)

[[  3. 127. 211.]
 [  5. 200. 254.]
 [ 29.   8.  10.]]


[[3, 127, 211]

[5, 200, 255]

[29, 8, 10]